In [1]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd

/dev/shm/venvs/airflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
recipe_df = pd.read_csv('/dev/shm/notebooks/recipe-context/merged_recipes.csv', on_bad_lines='skip')
print(recipe_df.shape)
recipe_df.sample(5)

(189926, 18)


/tmp/ipykernel_1571345/3952670458.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  recipe_df = pd.read_csv('/dev/shm/notebooks/recipe-context/merged_recipes.csv', on_bad_lines='skip')


,recipe_id,recipe_title,recipe_name,author_id,author_name,recipe_method,recipe_status,recipe_kind,time_taken,difficulty,recipe_url,portion,datePublished,food_img_url,ingredient,reviews,photo_reviews,comments
64919,6994256,[유아식]양배추어묵전 만들기,양배추어묵전,68314373,아가맘,부침,일상,밑반찬,30분이내,아무나,https://www.10000recipe.com/recipe/6994256,2인분,20221224233609,https://recipe1.ezmember.co.kr/cache/recipe/20...,"{'재료': [{'name': '사각어묵', 'amount': '1장'}, {'na...",0,0,0
77046,1974657,시원하고 매콤한 가지 고추냉국,가지고추냉국,cggirl1004,카푸치노,끓이기,일상,국/탕,15분이내,초급,https://www.10000recipe.com/recipe/1974657,2인분,20100710123800,https://recipe1.ezmember.co.kr/cache/recipe/20...,"{'재료': [{'name': '가지', 'amount': '2개'}, {'name...",0,0,1
102158,6901142,고추장아찌 무침 절임고추 무침 삭힌고추 무침,고추장아찌무침,hancy002,hancy002,무침,일상,밑반찬,15분이내,아무나,https://www.10000recipe.com/recipe/6901142,6인분이상,20181202183712,https://recipe1.ezmember.co.kr/cache/recipe/20...,"{'재료': [{'name': '삭힌 고추', 'amount': '30~40개 (고...",9,3,1
9960,6938593,돼지갈비강정~ 갈비 더이상 찜으로만 드시지말고 맛있게 튀겨 보세요^^,돼지갈비강정,homelover,홈러버,튀김,일상,메인반찬,60분이내,초급,https://www.10000recipe.com/recipe/6938593,4인분,20200807125649,https://recipe1.ezmember.co.kr/cache/recipe/20...,"{'재료': [{'name': '돼지갈비', 'amount': '600g'}, {'...",0,0,0
181069,6956196,양배추김치,양배추김치,cherimoya,체리모야,무침,일상,밑반찬,30분이내,초급,https://www.10000recipe.com/recipe/6956196,2인분,20210402214710,https://recipe1.ezmember.co.kr/cache/recipe/20...,"{'재료': [{'name': '양배추', 'amount': '200g'}, {'n...",0,0,0


In [3]:
# nan 거의 없음
recipe_df.recipe_name.isna().sum()/recipe_df.recipe_name.shape[0]

0.0006476206522540357

In [4]:
recipe_names = recipe_df.recipe_name.dropna().values.tolist()
len(recipe_names)

189803

In [5]:
recipe_names = recipe_names[:200]

변환

In [6]:
# 한국어 BERT 사전 훈련된 모델 로드
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
model = BertModel.from_pretrained('klue/bert-base')

In [7]:
# 텍스트를 토큰화하고 BERT 입력 형식에 맞게 변환
inputs = tokenizer(recipe_names, return_tensors="pt", padding=True, truncation=True, max_length=512)

# BERT 모델을 통해 텍스트 인코딩
with torch.no_grad():
    outputs = model(**inputs)

# 마지막 은닉층의 특징 벡터를 추출
last_hidden_states = outputs.last_hidden_state

# 첫 번째 토큰([CLS] 토큰)의 은닉 상태를 문장의 임베딩으로 사용
sentence_embedding = last_hidden_states[:, 0, :]

In [8]:
sentence_embedding.shape

torch.Size([200, 768])

faiss

In [10]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [11]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


In [12]:
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.175174  7.2076287 7.251163 ]
 [0.        6.323565  6.684582  6.799944 ]
 [0.        5.7964087 6.3917365 7.2815127]
 [0.        7.277905  7.5279875 7.6628447]
 [0.        6.763804  7.295122  7.368814 ]]
